<a href="https://colab.research.google.com/github/rabbitmetrics/langchain-agents-explained/blob/main/notebooks/langchain-agents-explained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shopify
import pandas as pd
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv())

In [ ]:
# Function that extracts data from Shopify's REST API

def get_data(object_name):
    api_session = shopify.Session(os.getenv('MERCHANT'),'2023-04',os.getenv('SHOPIFY_TOKEN'))
    shopify.ShopifyResource.activate_session(api_session)
    all_data=[]
    attribute=getattr(shopify,object_name)
    data=attribute.find(since_id=0, limit=250)
    for d in data:
        all_data.append(d)
    while data.has_next_page():
        data=data.next_page()
        for d in data:
            all_data.append(d)
    return all_data

In [ ]:
# Example: getting order data with the function
order_data=get_data('Order')

In [ ]:
# Import StructuredTool and ChatOpenAI needed for creating the agent based on GPT-4

from langchain.tools.base import StructuredTool
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(model_name="gpt-4",temperature=0.2)

In [ ]:
# Turn the get_data function into a new function that receives text and returns text and create a tool out of the new function

from typing import Optional

def get_shopify_insight(
    shopify_object: Optional[str] = None
) -> str:
    """Tool that counts the number of items for a given Shopify data object. Valid shopify_objects include "Customer", "Order", "Product" and "Webhook"."""
    object_name=(shopify_object or "Order")
    data= get_data(object_name)
    return str(len(data))

shopify_insights_tool = StructuredTool.from_function(get_shopify_insight)

In [ ]:
# We can now call the tool and count items returned as a string

shopify_insights_tool('Customer')

In [ ]:
# Create the agent based on the tool and the chatmodel

from langchain.agents import initialize_agent, AgentType
tools = [shopify_insights_tool]
agent_chain = initialize_agent(tools, 
                               chat, 
                               agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
                               verbose=True)

In [ ]:
# Run the agent

agent_chain("Count the number of orders in the Shopify store")